In [1]:
import pandas as pd
import sqlite3 

In [2]:
!pip install ipython-sql

Defaulting to user installation because normal site-packages is not writeable


In [3]:
#Joining dataset and reading csv to pandas dataframe:
df = pd.read_csv('cyclistic1.csv')

In [4]:
cnn = sqlite3.connect('-------.db')

In [5]:
df.to_sql('cyclisticdata', cnn, if_exists='replace')

1568271

In [6]:
%load_ext sql

In [7]:
%sql sqlite:///simsim.db

In [8]:
%%sql
SELECT *
FROM 
cyclisticdata
limit 10;

 * sqlite:///simsim.db
Done.


level_0,index,rideid,rideabletype,startedat,endedat,startstationname,startstationid,endstationname,endstationid,startlat,startlng,endlat,endlng,membercasual,month,dayofweek,starthour,endhour,durationmins
0,400,B23F6CCA8AC89F52,electricbike,2022-09-11T18:35:31,2022-09-11T18:45:57,Wood St & Chicago Ave,637,Wood St & Chicago Ave,637,41.89561,-87.672195,41.895634,-87.672069,casual,September,Sunday,18,18,10.433333333333334
1,964,2A39814848E1D54C,electricbike,2022-09-07T20:01:55,2022-09-07T20:19:24,California Ave & Milwaukee Ave,13084,California Ave & Milwaukee Ave,13084,41.922662333333335,-87.69710816666667,41.922695,-87.697153,casual,September,Wednesday,20,20,17.483333333333334
2,971,181A283DC7877573,electricbike,2022-09-08T17:00:30,2022-09-08T17:02:01,California Ave & Division St,13256,California Ave & Division St,13256,41.903035879,-87.697476506,41.903029,-87.697474,casual,September,Thursday,17,17,1.5166666666666666
3,1009,ACA9081DE3E5030A,classicbike,2022-09-10T14:14:09,2022-09-10T14:26:57,California Ave & Division St,13256,California Ave & Division St,13256,41.903029,-87.697474,41.903029,-87.697474,casual,September,Saturday,14,14,12.8
4,1638,192B8C5D116C3D76,classicbike,2022-09-03T19:58:59,2022-09-03T21:09:18,California Ave & Milwaukee Ave,13084,California Ave & Milwaukee Ave,13084,41.922695,-87.697153,41.922695,-87.697153,casual,September,Saturday,19,21,70.31666666666666
5,1647,D4CD3DF412980C2A,classicbike,2022-09-03T23:18:10,2022-09-03T23:34:21,Kedzie Ave & Lake St,KA1504000106,California Ave & Division St,13256,41.88460328501,-87.7063037436,41.903029,-87.697474,casual,September,Saturday,23,23,16.183333333333334
6,2201,FB3F59FA0A9FAD8D,classicbike,2022-09-23T21:55:04,2022-09-23T21:56:14,California Ave & Division St,13256,California Ave & Division St,13256,41.903029,-87.697474,41.903029,-87.697474,casual,September,Friday,21,21,1.1666666666666667
7,2237,A6FA0AAB756D6F13,classicbike,2022-09-08T18:37:19,2022-09-08T19:41:46,Cottage Grove Ave & 51st St,TA1309000067,Cottage Grove Ave & 51st St,TA1309000067,41.803038,-87.606615,41.803038,-87.606615,casual,September,Thursday,18,19,64.45
8,2581,0CDA5583BE8E3757,electricbike,2022-09-20T20:09:28,2022-09-20T20:16:09,California Ave & Milwaukee Ave,13084,California Ave & Division St,13256,41.922672,-87.697124,41.903029,-87.697474,casual,September,Tuesday,20,20,6.683333333333334
9,2594,32B5F2F44C16580C,classicbike,2022-09-10T10:26:16,2022-09-10T10:32:25,Wood St & Chicago Ave,637,Wood St & Chicago Ave,637,41.895634,-87.672069,41.895634,-87.672069,casual,September,Saturday,10,10,6.15


In [21]:
%%sql
/*Calculating total rides - counting ride id, total member rides and totalcasual as SUM(CASE WHEN = THEN 1 ELSE 0 END)*/
SELECT COUNT(rideid) AS totaltrips,
       SUM(CASE WHEN membercasual = 'member' THEN 1 ELSE 0 END) AS totalmember,
       SUM(CASE WHEN membercasual = 'casual' THEN 1 ELSE 0 END) AS totalcasual
FROM cyclisticdata;

 * sqlite:///simsim.db
Done.


totaltrips,totalmember,totalcasual
1568271,1054413,513858


In [ ]:
%%sql 
/*Count of nulls in rideid column*/
SELECT COUNT(CASE WHEN rideid IS NULL THEN 1 END) AS null_count
FROM cyclisticdata;

In [35]:
%%sql
/*SUBQUERY: of the 1568271 trips, 67percent were members and 33 were casual*/
SELECT totaltrips, totalmember, totalcasual,
        (CAST(totalcasual AS DECIMAL(10,2)) *100 / CAST(totaltrips AS DECIMAL(10,2))) AS casualpercentage,
        (CAST(totalmember AS DECIMAL(10,2)) *100/ CAST(totaltrips AS DECIMAL(10,2))) AS memberpercentage
FROM (
    SELECT COUNT(rideid) AS totaltrips,
           SUM(CASE WHEN membercasual = 'member' THEN 1 ELSE 0 END) AS totalmember,
           SUM(CASE WHEN membercasual = 'casual' THEN 1 ELSE 0 END) AS totalcasual
    FROM cyclisticdata
)AS subquery;

 * sqlite:///simsim.db
Done.


totaltrips,totalmember,totalcasual,casualpercentage,memberpercentage
1568271,1054413,513858,32,67


In [34]:
/*SUBQUERY: Avg ride duration: 14.27, avg casual duration: 19.99, avg member duration: 11.48*/
%%sql
SELECT
    (SELECT ROUND(AVG(durationmins), 2)
     FROM cyclisticdata) AS overallavg,
    membercasual,
    avgrideduration
FROM (
    SELECT membercasual, ROUND(AVG(durationmins), 2) AS avgrideduration
    FROM cyclisticdata
    WHERE membercasual  IN ('member', 'casual')
    GROUP BY membercasual
) AS subquery;

 * sqlite:///simsim.db
Done.


overallavg,membercasual,avgrideduration
14.27,casual,19.99
14.27,member,11.48


In [44]:
%%sql
/* casual max duration: 1438,  member max duration: 1436 */
SELECT  membercasual, ROUND(MAX(durationmins),0) AS maxduration
FROM cyclisticdata
GROUP BY membercasual;

 * sqlite:///simsim.db
Done.


membercasual,maxduration
casual,1438.0
member,1436.0


In [45]:
%%sql
/* casual min duration: 1,  member min duration: 1 */
SELECT membercasual, MIN(durationmins) AS minimduration
FROM cyclisticdata
GROUP BY membercasual
ORDER BY minimduration;

 * sqlite:///simsim.db
Done.


membercasual,minimduration
casual,1.0
member,1.0


In [58]:
%%sql
/* Subquery: busiest day of week for member: Thurs and casual: Sat*/
SELECT membercasual, dayofweek AS busiestweekday
FROM (
    SELECT DISTINCT membercasual, dayofweek,
           ROW_NUMBER() OVER (PARTITION BY membercasual ORDER BY COUNT(dayofweek) DESC) AS dowpopularity 
    FROM cyclisticdata
    GROUP BY membercasual, dayofweek
) AS subquery  
WHERE dowpopularity = 1
ORDER BY membercasual DESC;

 * sqlite:///simsim.db
Done.


membercasual,busiestweekday
member,Thursday
casual,Saturday


In [53]:
%%sql
/* Count of total rides by day sum of member and casual by day e.g. Thursday 243920 total trips, 172550 totalmember, 71370 total casual*/
    SELECT dayofweek, COUNT(rideid) AS totaltrips,
           SUM(CASE WHEN membercasual = 'member' THEN 1 ELSE 0 END) AS totalmember,
           SUM(CASE WHEN membercasual = 'casual' THEN 1 ELSE 0 END) AS totalcasual
    FROM cyclisticdata
    GROUP BY dayofweek
    ORDER BY totaltrips DESC;

 * sqlite:///simsim.db
Done.


dayofweek,totaltrips,totalmember,totalcasual
Thursday,243920,172550,71370
Saturday,236145,129661,106484
Wednesday,228137,167510,60627
Friday,227652,149797,77855
Tuesday,223980,168547,55433
Monday,214005,153586,60419
Sunday,194432,112762,81670


In [108]:
%%sql
/* Count of most popular start station visits and sum of visits by customer type*/
SELECT startstationname, 
       COUNT(CASE WHEN rideid = rideid AND startstationname = startstationname THEN 1 ELSE 0 END) AS startstationcount,
       SUM(CASE WHEN membercasual = 'member' THEN 1 ELSE 0 END) AS memberstationcount, 
       SUM(CASE WHEN membercasual = 'casual' THEN 1 ELSE 0 END) AS casualstationcount  
FROM cyclisticdata
GROUP BY startstationname
ORDER BY startstationcount DESC
LIMIT 3;

 * sqlite:///simsim.db
Done.


startstationname,startstationcount,memberstationcount,casualstationcount
Streeter Dr & Grand Ave,17931,4474,13457
Ellis Ave & 60th St,13031,10704,2327
University Ave & 57th St,12501,10291,2210


In [88]:
%%sql
/* Subquery: most popular start and end station by customer type -memberstart: Ellis av & 60th st, memberend: university av & 57th st, casualstart streeter dr & grand av, casualend streeter dr & grand av */
SELECT membercasual,
       startstationname AS busieststartstation,
       endstationname AS busiestendstation
FROM (
  SELECT membercasual,
         startstationname,
         endstationname,
         ROW_NUMBER() OVER (PARTITION BY membercasual ORDER BY COUNT(*) DESC) AS stationpopularity
  FROM cyclisticdata
  GROUP BY membercasual, startstationname, endstationname
) AS station_counts
WHERE stationpopularity = 1
ORDER BY membercasual DESC;

 * sqlite:///simsim.db
Done.


membercasual,busieststartstation,busiestendstation
member,Ellis Ave & 60th St,University Ave & 57th St
casual,Streeter Dr & Grand Ave,Streeter Dr & Grand Ave


In [107]:
%%sql
/*Practicing without partition by/subquery: 5pm is the most popular time, member: 4pm and 5pm yet member has a 9am peak, casual: 4pm and 5pm*/
SELECT membercasual, starthour, COUNT(starthour) AS popstarthour
FROM cyclisticdata
GROUP BY membercasual, starthour
ORDER BY  popstarthour DESC;


 * sqlite:///simsim.db
Done.


membercasual,starthour,popstarthour
member,17,112889
member,16,97404
member,18,85312
member,15,74038
member,8,73577
member,7,60164
member,12,59889
member,13,59468
member,14,58940
member,19,57418
